In [96]:
import numpy as np
import os
import librosa
import IPython.display as ipd
from scipy import stats

# throw error if files exists


raw_data_dir = 'E:\dataset'
pwd = os.getcwd()
seed = 7
limit = 50

# stft params
sr = 44100
# sr = 22050
n_fft = 1024
h = int(n_fft/2)
# direcoties
noised_data_dir = os.path.join(raw_data_dir, 'noisyreverb')
clean_data_dir = os.path.join(raw_data_dir , 'clean')
# print(noised_data_dir)
# data = np.empty(shape=(100,2),dtype=str)
selected_files = {}
np.random.seed(seed)
while len(selected_files) < limit:
    # collect 100 random unique files from noisyreverb
    # print(clean_data_dir.)
    max = len(os.listdir(noised_data_dir))
    rand_index = np.random.randint(0,max)
    while rand_index in selected_files:
        rand_index = np.random.randint(0,max)
    random_file = os.listdir(noised_data_dir)[rand_index]
    assert random_file in os.listdir(clean_data_dir)
    selected_files[rand_index] = random_file
# print(selected_files.values())
wav_files = list(selected_files.values())
X = np.empty(shape=(len(wav_files)),dtype=object)
Y = np.empty(shape=(len(wav_files)),dtype=object)
# print(wav_files)
test = []
for n, file_name in enumerate(wav_files):
    clean_file = os.path.join(clean_data_dir, file_name)
    noise_file = os.path.join(noised_data_dir, file_name)
    clean_signal, _ = librosa.load(clean_file, sr=sr)
    noise_signal, _ = librosa.load(noise_file, sr=sr)
    assert clean_signal.shape == noise_signal.shape
    
    clean_spec = librosa.stft(clean_signal, n_fft=n_fft, hop_length=h)
    noised_spec = librosa.stft(noise_signal, n_fft=n_fft, hop_length=h)
    assert clean_spec.shape == noised_spec.shape
        
    if n == 10:
        print(file_name)
        test = clean_spec, noised_spec
    # clean_spec = librosa.util.normalize((clean_spec))
    # noised_spec = librosa.util.normalize(noised_spec)

    # break
    X[n] = noised_spec
    Y[n] = clean_spec
    # print(Y[n].shape)
    # break
np.save(f'data\X_{seed}_{limit}.npy', X)
np.save(f'data\Y_{seed}_{limit}.npy', Y)
    

p250_351.wav


In [99]:
import IPython.display as ipd
ipd.Audio(librosa.istft(test[0], n_fft=n_fft, hop_length=h), rate=48000)

In [ ]:
import numpy as np
import tensorflow as tf
dim_0 = 200
dim_1 = 513
X = np.empty(shape=(200,), dtype=object)
pyX = []
for i in range(200):
    val = np.random.rand(dim_1, np.random.randint(200,400))+1j
    X[i] = val
    pyX.append(val)
    # print(X[i].dtype)
# grab the size of the 2nd dimension of each sample and store it in a list
rowbreaks = [x.shape[1] for x in X]

In [ ]:
X_tf = [tf.constant(x) for x in X]
max_length = tf.reduce_max([tf.shape(x)[0] for x in X_tf])
X_tf = [tf.pad(x, [[0, max_length - tf.shape(x)[0]], [0, 0]]) for x in X_tf]
row_lengths = [[x.shape[1]] for x in X_tf]
ragged_tensor = tf.RaggedTensor.from_nested_row_lengths(X_tf, nested_row_lengths=row_lengths, validate=True)

In [ ]:
# First, create a list of tensors with the desired shapes
X_tf = [tf.random.normal((513, dim_2)) for dim_2 in range(200, 400)]

# Then, use `tf.ragged.stack` to stack the tensors into a ragged tensor
ragged_tensor = tf.ragged.stack(X_tf)

# Finally, verify that the ragged tensor has the desired shape
# assert ragged_tensor.shape == (200, 513, None)
ragged_tensor.shape

In [ ]:
## None of these work
tr = [tf.constant(x) for x in X] # TensorShape([513, (dim_2)])
# tx = tf.ragged.constant(pylist=tr, inner_shape=[513,None], ragged_rank=1,) # Doesn't terminate
# tx = tf.ragged.constant(pylist=tr, ragged_rank=1) # Doesn't Terminate 
# tx = tf.ragged.constant(pylist=tr, inner_shape=[513,None]) # Doesn't terminate
# tx = tf.RaggedTensor.from_row_lengths(values=tr, row_lengths=rowbreaks) # values[0].shape = [513,351] != values[1].shape = [513,202]
# tx = tf.RaggedTensor.from_uniform_row_length(values=[tf.transpose(t) for t in tr], uniform_row_length=513, nrows=200 ) # shapes neq too in value
# tx = tf.RaggedTensor.from_uniform_row_length(values=[tf.transpose(t) for t in tr], uniform_row_length=513)# neq shapes in "value" 
# tx = tf.RaggedTensor.from_tensor(tensor=tf.stack(tr, axis=0), ragged_rank=1) # neq shapes trying to stack
# tx = tf.RaggedTensor.from_tensor(tensor=tf.stack(tr), lengths=rowbreaks) # Dst tensor is not initialized
# tx = tf.RaggedTensor.from_tensor(tf.ragged.stack(tr, axis=0)) # object of type 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_tensor(tensor=tf.ragged.stack(tr), lengths=rowbreaks) # Dst tensor is not initialized
# tx = tf.RaggedTensor.from_tensor(tr, ragged_rank=1) # neq shapes in Tensor

In [ ]:
# tr = [tf.ragged.constant(tf.stack(x)) for x in X] # Crashed after 4 minutes. Scalar tensor has no `len()`
tr = [tf.ragged.constant(x) for x in X] #
#If a ragged tensor can be defined using one or more ragged tensors with a uniform row length. Then we should be able to make 513xNone Ragged tensors for the stfts by stacking each frequency bin as row which should be a ragged tensor itself.

In [ ]:

def gen():
    for x in X:
        x_ragged = tf.ragged.stack(x.tolist(), axis=0)
        print(x_ragged.shape, x_ragged.dtype, type(x_ragged))
        yield x
dataset = tf.data.Dataset.from_generator(generator=gen, output_signature=tf.TensorSpec(shape=(513, None), dtype=np.complex128))
dataset.element_spec

In [ ]:
# tx = tf.RaggedTensor.from_tensor(tf.stack(tr, axis=0), ragged_rank=1) # neq shapes trying to stack
# tx = tf.RaggedTensor.from_tensor(tf.ragged.stack(tr, axis=0)) # object of type 'RaggedTensor' has no len()
# tx = tf.ragged.constant(pylist=tr, inner_shape=[None,513]) # 'RaggedTensor' has no len()
# tx = tf.ragged.constant(pylist=tr) # 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_row_lengths(tr,row_lengths=[x.shape[1] for x in tr]) # Fails converting None to Tensor
# tx = tf.RaggedTensor.from_row_lengths(tr,row_lengths=rowbreaks) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_nested_row_lengths(tr, nested_row_lengths=[tf.fill(dims=[513,],value=l ) for l in rowbreaks]) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_uniform_row_length([tf.transpose(t) for t in tr], uniform_row_length=513) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_tensor(tensor=tf.stack(tr), ragged_rank=1) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_tensor(tensor=tf.stack(tr)) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_tensor(tf.stack(tr), lengths=rowbreaks) # Fails: 'RaggedTensor' has no len()
mid = tf.ragged.stack(tr, axis=0)
# tx = tf.RaggedTensor.from_tensor(mid, ragged_rank=1) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_tensor(mid) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_tensor(mid, lengths=rowbreaks) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_row_lengths(mid,row_lengths=rowbreaks) # Fails: 'RaggedTensor' has no len()
# tx = tf.RaggedTensor.from_nested_row_lengths(mid, nested_row_lengths=[tf.fill(dims=[513,],value=l ) for l in rowbreaks]) # Fails: Differing dtypes
# tx = tf.RaggedTensor.from_uniform_row_length([tf.transpose(t) for t in mid], uniform_row_length=513) # Fails: 'RaggedTensor' has no len()
# tx = tf.ragged.constant(mid, inner_shape=[513, None]) # pylist may not be a RaggedTensor

In [ ]:
## This works but the shape is not proper
import tensorflow as tf
tr = [tf.constant(x) for x in X] # TensorShape([513, (dim_2)])
tx = tf.ragged.stack(tr)# TensorShape([200, 513, (dim_2)])
print(tx.shape)